In [ ]:
# Install Whisper
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg -y

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-y9rk9n8i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-y9rk9n8i
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=bb6ae4affffd02c2feee9c4d42232cc58ff1da417e382865dbccbf14b972486a
  Stored in directory: /tmp/pip-ephem-wheel-cache-c2idb9xx/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path
import whisper
import torch
from google.colab import drive

# Format timestamps for SRT subtitles
def format_timestamp(seconds: float) -> str:
    millis = int((seconds % 1) * 1000)
    seconds = int(seconds)
    mins, sec = divmod(seconds, 60)
    hrs, mins = divmod(mins, 60)
    return f"{hrs:02d}:{mins:02d}:{sec:02d},{millis:03d}"

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔥 Using device:", device)

# Set your input and output directories here
input_folder = "/content/drive/MyDrive/Commercial Horticulture English"     # TODO: Change this
output_folder = "/content/drive/MyDrive/D1 ASR"   # TODO: Change this

# Load Whisper model
model = whisper.load_model("medium").to(device)

# Process all .mp4 files in the input folder (including subfolders)
for root, _, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".mp4"):
            input_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, input_folder)
            output_dir = os.path.join(output_folder, relative_path)
            os.makedirs(output_dir, exist_ok=True)

            print(f"\n🎙️ Transcribing: {file}")

            try:
                result = model.transcribe(
                    input_path,
                    language="en",
                    task="transcribe",
                    fp16=torch.cuda.is_available()
                )

                # Save plain text transcript
                text_path = os.path.join(output_dir, f"{Path(file).stem}.txt")
                with open(text_path, "w", encoding="utf-8") as f:
                    f.write(result["text"])

                # Save subtitles in SRT format
                srt_path = os.path.join(output_dir, f"{Path(file).stem}.srt")
                with open(srt_path, "w", encoding="utf-8") as f:
                    for i, segment in enumerate(result["segments"], start=1):
                        f.write(
                            f"{i}\n"
                            f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n"
                            f"{segment['text'].strip()}\n\n"
                        )

                print(f"✅ Finished: {file}")

            except Exception as e:
                print(f"❌ Error with {file}: {e}")


🔥 Using device: cuda


100%|█████████████████████████████████████| 1.42G/1.42G [00:50<00:00, 30.5MiB/s]



🎙️ Transcribing: Chapter 4 - Responsible use of Pesticides and Natural Alternatives.mp4
✅ Finished: Chapter 4 - Responsible use of Pesticides and Natural Alternatives.mp4

🎙️ Transcribing: Chapter 4B - Organic methods like companion planting and natural predators.mp4
✅ Finished: Chapter 4B - Organic methods like companion planting and natural predators.mp4

🎙️ Transcribing: Chapter 4A - Responsible use of pesticides and natural alternatives.mp4
✅ Finished: Chapter 4A - Responsible use of pesticides and natural alternatives.mp4

🎙️ Transcribing: Chapter 2 - Major Fruit Crops of the Region.mp4
✅ Finished: Chapter 2 - Major Fruit Crops of the Region.mp4

🎙️ Transcribing: Chapter 2F - Dry land horticulture & its management practices.mp4
✅ Finished: Chapter 2F - Dry land horticulture & its management practices.mp4

🎙️ Transcribing: Chapter 2B - Major flower crops of the region, floriculture and landscape gardening maintenance of lawns.mp4
✅ Finished: Chapter 2B - Major flower crops of the 